In [7]:
from kiteconnect import KiteConnect
import pandas as pd
from selenium import webdriver
import time
import os
import datetime as dt
pd.options.mode.chained_assignment = None
import numpy as np

In [2]:
token_path = 'api_key.txt'
key_secret = open(token_path,'r').read().split()   #gives a list object
kite = KiteConnect(api_key = key_secret[0])
#virtually start the chromedriver -  don't make the window pop up
service = webdriver.chrome.service.Service('C:/Users/crypt/.wdm/drivers/chromedriver/win32/83.0.4103.39/chromedriver.exe')
service.start()
#don't render the browser, save some RAM
options = webdriver.ChromeOptions()
options.add_argument('--headless')
#just aggregating all the capabilities like headless, memoryless etc
options = options.to_capabilities()
#start the driver, pass the options 
driver = webdriver.Remote(service.service_url,options)
driver.get(kite.login_url())
driver.implicitly_wait(2)
#fint the element in the webpage by the HTML path
username = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div/div[2]/form/div[1]/input')
password = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div/div[2]/form/div[2]/input')
username.send_keys(key_secret[2])
password.send_keys(key_secret[3])
driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div/div[2]/form/div[4]/button').click()
pin = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div/div[2]/form/div[2]/div/input')
pin.send_keys(key_secret[4])
driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div/div[2]/form/div[3]/button').click()
time.sleep(2)
request_token = driver.current_url.split('=')[1].split('&action')[0]
with open('request_token.txt','w') as file:
    file.write(request_token)
driver.quit()

#Use request_token and get_access_token
request_token = open('request_token.txt','r').read().split()
key_secret = open('api_key.txt','r').read().split()
kite = KiteConnect(api_key = key_secret[0])
data = kite.generate_session(request_token[0],api_secret = key_secret[1])
with open('access_token.txt','w') as file:
    file.write(data['access_token'])
    
# Set access token and you're all set!!!!!!!!!!   
kite.set_access_token(open('access_token.txt','r').read().split()[0])

In [8]:
def instrumentLookup(instrument_df,symbol):
    """Looks up instrument token for a given script from instrument dump"""
    try:
        return instrument_df[instrument_df.tradingsymbol==symbol].instrument_token.values[0]
    except:
        return -1


def fetchOHLC(ticker,interval,duration):
    """extracts historical data and outputs in the form of dataframe"""
    instrument = instrumentLookup(instrument_df,ticker)
    data = pd.DataFrame(kite.historical_data(instrument,dt.date.today()-dt.timedelta(duration), dt.date.today(),interval))
    data.set_index("date",inplace=True)
    return data



    
def adx(ohlc):
    "function to calculate ADX"
    n=14
    df2 = ohlc.copy()
    df2['H-L']=abs(df2['high']-df2['low'])
    df2['H-PC']=abs(df2['high']-df2['close'].shift(1))
    df2['L-PC']=abs(df2['low']-df2['close'].shift(1))
    df2['TR']=df2[['H-L','H-PC','L-PC']].max(axis=1,skipna=False)
    df2['DMplus']=np.where((df2['high']-df2['high'].shift(1))>(df2['low'].shift(1)-df2['low']),df2['high']-df2['high'].shift(1),0)
    df2['DMplus']=np.where(df2['DMplus']<0,0,df2['DMplus'])
    df2['DMminus']=np.where((df2['low'].shift(1)-df2['low'])>(df2['high']-df2['high'].shift(1)),df2['low'].shift(1)-df2['low'],0)
    df2['DMminus']=np.where(df2['DMminus']<0,0,df2['DMminus'])
    TRn = []
    DMplusN = []
    DMminusN = []
    TR = df2['TR'].tolist()
    DMplus = df2['DMplus'].tolist()
    DMminus = df2['DMminus'].tolist()
    for i in range(len(df2)):
        if i < n:
            TRn.append(np.NaN)
            DMplusN.append(np.NaN)
            DMminusN.append(np.NaN)
        elif i == n:
            TRn.append(df2['TR'].rolling(n).sum().tolist()[n])
            DMplusN.append(df2['DMplus'].rolling(n).sum().tolist()[n])
            DMminusN.append(df2['DMminus'].rolling(n).sum().tolist()[n])
        elif i > n:
            TRn.append(TRn[i-1] - (TRn[i-1]/n) + TR[i])
            DMplusN.append(DMplusN[i-1] - (DMplusN[i-1]/n) + DMplus[i])
            DMminusN.append(DMminusN[i-1] - (DMminusN[i-1]/n) + DMminus[i])
    df2['TRn'] = np.array(TRn)
    df2['DMplusN'] = np.array(DMplusN)
    df2['DMminusN'] = np.array(DMminusN)
    df2['DIplusN']=100*(df2['DMplusN']/df2['TRn'])
    df2['DIminusN']=100*(df2['DMminusN']/df2['TRn'])
    df2['DIdiff']=abs(df2['DIplusN']-df2['DIminusN'])
    df2['DIsum']=df2['DIplusN']+df2['DIminusN']
    df2['DX']=100*(df2['DIdiff']/df2['DIsum'])
    ADX = []
    DX = df2['DX'].tolist()
    for j in range(len(df2)):
        if j < 2*n-1:
            ADX.append(np.NaN)
        elif j == 2*n-1:
            ADX.append(df2['DX'][j-n+1:j+1].mean())
        elif j > 2*n-1:
            ADX.append(((n-1)*ADX[j-1] + DX[j])/n)
    df2['ADX']=np.array(ADX)
    df3 = df2.loc[:,'DIdiff':'DX']
    return df3



def Expansion_Breakouts(ohlc_1day,ticker):
    
    if ohlc_1day["high"][-1]>=ohlc_1day["high"][-59:-1].max():
        if ohlc_1day["high"][-1]-ohlc_1day["low"][-1]>=(ohlc_1day["high"][-10:-1]-ohlc_1day["low"][-10:-1]).max():
            print("strategy 1, expansion breakout = buy:",ticker)
            return "b"
            
    if ohlc_1day["low"][-1]<=ohlc_1day["low"][-59:-1].min():
        if ohlc_1day["high"][-1]-ohlc_1day["low"][-1]>=(ohlc_1day["high"][-10:-1]-ohlc_1day["low"][-10:-1]).max():
            print("strategy 1, expansion breakout = sell:",ticker)
            return "s"
    else:
        print("strategy 1, expansion breakout = NOTHING:",ticker)
        
def OTT_Pullback(ohlc_1day,ticker):
    adx1=adx(ohlc_1day)
    if adx1['DX'][-3]>30:
        if adx1['DIsum'][-3]>adx1['DIdiff'][-3]:
            if (ohlc_1day['low'][-3]>ohlc_1day['low'][-2] and ohlc_1day['low'][-2]>ohlc_1day['low'][-1]) or (ohlc_1day['low'][-3]>ohlc_1day['low'][-2] and ohlc_1day['low'][-2]<ohlc_1day['low'][-1]):
                print("RARE strategy 2, 1-2-3 pullback = buy:",ticker)
                return "b"
            
    if adx1['DX'][-3]>30:
        if adx1['DIdiff'][-3]>adx1['DIsum'][-3]:
            if (ohlc_1day['high'][-3]<ohlc_1day['high'][-2] and ohlc_1day['high'][-2]<ohlc_1day['high'][-1]) or (ohlc_1day['high'][-3]<ohlc_1day['high'][-2] and ohlc_1day['high'][-2]>ohlc_1day['high'][-1]):
                print("RARE strategy 2, 1-2-3 pullback = sell:",ticker)
                return "s"
    else:
        print("strategy 2, OTT pullback = NOTHING:",ticker)

def Expansion_pivots(ohlc_1day,ticker):
    ma = ohlc_1day["close"].rolling(50).mean()
    if ohlc_1day["high"][-1]-ohlc_1day["low"][-1]>=(ohlc_1day["high"][-10:-1]-ohlc_1day["low"][-10:-1]).max():
        if ohlc_1day["high"][-2]<=ma[-2] and ohlc_1day["low"][-1]>ma[-1]:
            print("strategy 3, expansion pivots = buy:",ticker)
            return "b"
        
    if ohlc_1day["high"][-1]-ohlc_1day["low"][-1]>=(ohlc_1day["high"][-10:-1]-ohlc_1day["low"][-10:-1]).max():
        if ohlc_1day["low"][-2]>=ma[-2] and ohlc_1day["high"][-1]<ma[-1]:
            print("strategy 3, expansion pivots = sell:",ticker)
            return "s"
        
    else:
        print("strategy 3, expansion pivots = NOTHING:",ticker)
        
        
def One_eighty(ohlc_1day,ticker):
    
    ma_50 = ohlc_1day["close"].rolling(50).mean()
    ma_10 = ohlc_1day["close"].rolling(10).mean()
    
    if ((ohlc_1day["high"][-2]-ohlc_1day["open"][-2])/(ohlc_1day["high"][-2]-ohlc_1day["low"][-2]))>=0.7:
        if ((ohlc_1day["high"][-1]-ohlc_1day["open"][-1])/(ohlc_1day["high"][-1]-ohlc_1day["low"][-1]))>=0.3:
            if(ohlc_1day["low"][-1]>ma_50[-1] and ohlc_1day["low"][-1]>ma_10[-1]):
                print("strategy 4, 180 deg = buy:",ticker)
                return "b"
                
    if ((ohlc_1day["high"][-2]-ohlc_1day["open"][-2])/(ohlc_1day["high"][-2]-ohlc_1day["low"][-2]))>=0.3:
        if ((ohlc_1day["high"][-1]-ohlc_1day["open"][-1])/(ohlc_1day["high"][-1]-ohlc_1day["low"][-1]))>=0.7:
            if(ohlc_1day["high"][-1]<ma_50[-1] and ohlc_1day["high"][-1]<ma_10[-1]):
                print("strategy 4, 180 deg = sell:",ticker)
                return "s"
                
                
    if ((ohlc_1day["high"][-2]-ohlc_1day["open"][-2])/(ohlc_1day["high"][-2]-ohlc_1day["low"][-2]))>=0.7:
        if ((ohlc_1day["high"][-1]-ohlc_1day["open"][-1])/(ohlc_1day["high"][-1]-ohlc_1day["low"][-1]))>=0.3:
            if(ohlc_1day["low"][-1]>ma_50[-1] and ohlc_1day["low"][-1]>ma_10[-1]):
                if ohlc_1day["high"][-1]>=ohlc_1day["high"][-59:-1].max():
                    print("strategy 4, new 60 day high add more position 180 deg = buy:",ticker)
                    return "b"
                         
    if ((ohlc_1day["high"][-2]-ohlc_1day["open"][-2])/(ohlc_1day["high"][-2]-ohlc_1day["low"][-2]))>=0.3:
        if ((ohlc_1day["high"][-1]-ohlc_1day["open"][-1])/(ohlc_1day["high"][-1]-ohlc_1day["low"][-1]))>=0.7:
            if(ohlc_1day["high"][-1]<ma_50[-1] and ohlc_1day["high"][-1]<ma_10[-1]):
                if ohlc_1day["low"][-1]<=ohlc_1day["low"][-59:-1].min():
                    print("strategy 4,new 60 day low add more position  180 deg = sell:",ticker)
                    return "s"
    else:
        print("strategy 4, 180 deg = NOTHING:",ticker)
        
        
def Gilligan_Island(ohlc_1day,ticker):
    if ohlc_1day["high"][-1]<=ohlc_1day["low"][-59:-1].min():
        if ((ohlc_1day["close"][-1]-ohlc_1day["low"][-1])/(ohlc_1day["high"][-1]-ohlc_1day["low"][-1]))>=0.45:
             print("strategy 5, gulligan island = buy:",ticker)
             return "b"
         
    if ohlc_1day["low"][-1]>=ohlc_1day["high"][-59:-1].max():
        if ((ohlc_1day["high"][-1]-ohlc_1day["open"][-1])/(ohlc_1day["high"][-1]-ohlc_1day["low"][-1]))>=0.45:
             print("strategy 5, gulligan island = sell:",ticker)
             return "s"
    else:
        print("strategy 5, gulligan island = NOTHING:",ticker)
        
        
def Boomer(ohlc_1day,ticker):
    adx2=adx(ohlc_1day)
    if adx2['DX'][-3]>30:
        if adx2['DIsum'][-3]>adx2['DIdiff'][-3]:
            if ohlc_1day["high"][-3]>ohlc_1day["high"][-2] and ohlc_1day["high"][-2]>ohlc_1day["high"][-1]:
                if ohlc_1day["low"][-3]<ohlc_1day["low"][-2] and ohlc_1day["low"][-2]<ohlc_1day["low"][-1]:
                    print("strategy 6, Boomer = buy:",ticker)
                    return "b"
                    
        if adx2['DIdiff'][-3]>adx2['DIsum'][-3]:
            if ohlc_1day["high"][-3]>ohlc_1day["high"][-2] and ohlc_1day["high"][-2]>ohlc_1day["high"][-1]:
                if ohlc_1day["low"][-3]<ohlc_1day["low"][-2] and ohlc_1day["low"][-2]<ohlc_1day["low"][-1]:
                    print("strategy 6, Boomer = sell:",ticker)
                    return "s"
        
    else:
        print("strategy 6, Boomer = NOTHING:",ticker)
        
        
def Slingshot(ohlc_1day,ticker):
    if ohlc_1day["high"][-2]>=ohlc_1day["high"][-60:-2].max():
        if ohlc_1day["high"][-1]<ohlc_1day["high"][-2]:
            print("strategy 7, Slingshot = buy:",ticker)
            return "b"
        
    if ohlc_1day["low"][-2]<=ohlc_1day["low"][-60:-2].min():
        if ohlc_1day["low"][-1]>ohlc_1day["low"][-2]:
            print("strategy 7, Slingshot = sell:",ticker)
            return "s"
    else:
        print("strategy 7, Slingshot = NOTHING:",ticker)

def Whoops(ohlc_1day,ticker):
    ma_50 = ohlc_1day["close"].rolling(50).mean()
    ma_10 = ohlc_1day["close"].rolling(10).mean()
    if ohlc_1day["high"][-1]<ma_10[-1] and ohlc_1day["high"][-1]<ma_50[-1]:
        if ohlc_1day["close"][-2]>ohlc_1day["high"][-1]:
            print("strategy 8, Whoops = sell:",ticker)
            return "s"
        
    else:
        print("strategy 8, Whoops = NOTHING:",ticker)
        
def Lizards(ohlc_1day,ticker):
    if ((ohlc_1day["close"][-1]-ohlc_1day["low"][-1])/(ohlc_1day["high"][-1]-ohlc_1day["low"][-1]))>=0.7:
        if ohlc_1day["low"][-1]<=ohlc_1day["low"][-11:-1].min():
             print("strategy 9, Lizards = buy:",ticker)
             return "b"
        
    if ((ohlc_1day["high"][-1]-ohlc_1day["open"][-1])/(ohlc_1day["high"][-1]-ohlc_1day["low"][-1]))>=0.7:
        if ohlc_1day["high"][-1]>=ohlc_1day["high"][-11:-1].max():
             print("strategy 9, Lizards = sell:",ticker)
             return "s"
             
    else:
        print ("strategy 9, Lizards = NOTHING:",ticker)
             
             
def Compile(ohlc_1day,ticker):
    buy=0
    sell=0
    EB=Expansion_Breakouts(ohlc_1day,ticker)
    if EB == "b":
        buy+=1
    if EB == "s":
        sell+=1
    
    OTP = OTT_Pullback(ohlc_1day,ticker)
    if OTP == "b":
        buy+=1
    if OTP == "s":
        sell+=1
    
    EP = Expansion_pivots(ohlc_1day,ticker)
    if EP == "b":
        buy+=1
    if EP == "s":
        sell+=1
    
    OE = One_eighty(ohlc_1day,ticker)
    if OE == "b":
        buy+=1
    if OE == "s":
        sell+=1
    
    GI = Gilligan_Island(ohlc_1day,ticker)
    if GI == "b":
        buy+=1
    if GI == "s":
        sell+=1
      
    BO = Boomer(ohlc_1day,ticker)
    if BO == "b":
        buy+=1
    if BO == "s":
        sell+=1
     
    SS = Slingshot(ohlc_1day,ticker) 
    if SS == "b":
        buy+=1
    if SS == "s":
        sell+=1
    
    WH =Whoops(ohlc_1day,ticker)
    if WH == "b":
        buy+=1
    if WH == "s":
        sell+=1
    
    LZ = Lizards(ohlc_1day,ticker)
    if LZ == "b":
        buy+=1
    if LZ == "s":
        sell+=1
        
    print(buy)
    print(sell)
    
   
    

def main():
    tickers = ["ZEEL","WIPRO","VEDL","ULTRACEMCO","UPL","TITAN","TECHM","TATASTEEL",
           "TATAMOTORS","TCS","SUNPHARMA","SBIN","SHREECEM","RELIANCE","POWERGRID",
           "ONGC","NESTLEIND","NTPC","MARUTI","M&M","LT","KOTAKBANK","JSWSTEEL","INFY",
           "INDUSINDBK","IOC","ITC","ICICIBANK","HDFC","HINDUNILVR","HINDALCO",
           "HEROMOTOCO","HDFCBANK","HCLTECH","GRASIM","GAIL","EICHERMOT","DRREDDY",
           "COALINDIA","CIPLA","BRITANNIA","INFRATEL","BHARTIARTL","BPCL","BAJAJFINSV",
           "BAJFINANCE","BAJAJ-AUTO","AXISBANK","ASIANPAINT","ADANIPORTS","IDEA",
           "MCDOWELL-N","UBL","NIACL","SIEMENS","SRTRANSFIN","SBILIFE","PNB",
           "PGHH","PFC","PEL","PIDILITIND","PETRONET","PAGEIND","OFSS","NMDC","NHPC",
           "MOTHERSUMI","MARICO","LUPIN","L&TFH","INDIGO","IBULHSGFIN","ICICIPRULI",
           "ICICIGI","HINDZINC","HINDPETRO","HAVELLS","HDFCLIFE","HDFCAMC","GODREJCP",
           "GICRE","DIVISLAB","DABUR","DLF","CONCOR","COLPAL","CADILAHC","BOSCHLTD",
           "BIOCON","BERGEPAINT","BANKBARODA","BANDHANBNK","BAJAJHLDNG",
           "AUROPHARMA","ASHOKLEY","AMBUJACEM","ADANITRANS","ACC",
           "WHIRLPOOL","WABCOINDIA","VOLTAS","VINATIORGA","VBL","VARROC","VGUARD",
           "UNIONBANK","UCOBANK","TRENT","TORNTPOWER","TORNTPHARM","THERMAX","RAMCOCEM",
           "TATAPOWER","TATACONSUM","TVSMOTOR","TTKPRESTIG","SYNGENE","SYMPHONY",
           "SUPREMEIND","SUNDRMFAST","SUNDARMFIN","SUNTV","STRTECH","SAIL","SOLARINDS",
           "SHRIRAMCIT","SCHAEFFLER","SANOFI","SRF","SKFINDIA","SJVN","RELAXO",
           "RAJESHEXPO","RECLTD","RBLBANK","QUESS","PRESTIGE","POLYCAB","PHOENIXLTD",
           "PFIZER","PNBHOUSING","PIIND","OIL","OBEROIRLTY","NAM-INDIA","NATIONALUM",
           "NLCINDIA","NBCC","NATCOPHARM","MUTHOOTFIN","MPHASIS","MOTILALOFS","MINDTREE",
           "MFSL","MRPL","MANAPPURAM","MAHINDCIE","M&MFIN","MGL","MRF","LTI","LICHSGFIN",
           "LTTS","KANSAINER","KRBL","JUBILANT","JUBLFOOD","JINDALSTEL","JSWENERGY",
           "IPCALAB","NAUKRI","IGL","IOB","INDHOTEL","INDIANB","IBVENTURES","IDFCFIRSTB",
           "IDBI","ISEC","HUDCO","HONAUT","HAL","HEXAWARE","HATSUN","HEG","GSPL",
           "GUJGASLTD","GRAPHITE","GODREJPROP","GODREJIND","GODREJAGRO","GLENMARK",
           "GLAXO","GILLETTE","GMRINFRA","FRETAIL","FCONSUMER","FORTIS","FEDERALBNK",
           "EXIDEIND","ESCORTS","ERIS","ENGINERSIN","ENDURANCE","EMAMILTD","EDELWEISS",
           "EIHOTEL","LALPATHLAB","DALBHARAT","CUMMINSIND","CROMPTON","COROMANDEL","CUB",
           "CHOLAFIN","CHOLAHLDNG","CENTRALBK","CASTROLIND","CANBK","CRISIL","CESC",
           "BBTC","BLUEDART","BHEL","BHARATFORG","BEL","BAYERCROP","BATAINDIA",
           "BANKINDIA","BALKRISIND","ATUL","ASTRAL","APOLLOTYRE","APOLLOHOSP",
           "AMARAJABAT","ALKEM","APLLTD","AJANTPHARM","ABFRL","ABCAPITAL","ADANIPOWER",
           "ADANIGREEN","ADANIGAS","ABBOTINDIA","AAVAS","AARTIIND","AUBANK","AIAENG","3MINDIA"]
    for ticker in tickers:
        ohlc_1day = fetchOHLC(ticker,'day',100) 
        Compile(ohlc_1day,ticker)  
   

In [9]:
if __name__ == "__main__":
    main()


strategy 1, expansion breakout = NOTHING: ZEEL
strategy 2, OTT pullback = NOTHING: ZEEL
strategy 3, expansion pivots = NOTHING: ZEEL
strategy 4, 180 deg = NOTHING: ZEEL
strategy 5, gulligan island = NOTHING: ZEEL
strategy 6, Boomer = NOTHING: ZEEL
strategy 7, Slingshot = NOTHING: ZEEL
strategy 8, Whoops = NOTHING: ZEEL
strategy 9, Lizards = NOTHING: ZEEL
0
0
strategy 1, expansion breakout = NOTHING: WIPRO
strategy 3, expansion pivots = NOTHING: WIPRO
strategy 5, gulligan island = NOTHING: WIPRO
strategy 7, Slingshot = NOTHING: WIPRO
strategy 8, Whoops = NOTHING: WIPRO
strategy 9, Lizards = NOTHING: WIPRO
0
0
strategy 1, expansion breakout = NOTHING: VEDL
RARE strategy 2, 1-2-3 pullback = buy: VEDL
strategy 3, expansion pivots = NOTHING: VEDL
strategy 4, 180 deg = NOTHING: VEDL
strategy 5, gulligan island = NOTHING: VEDL
strategy 7, Slingshot = NOTHING: VEDL
strategy 8, Whoops = NOTHING: VEDL
strategy 9, Lizards = NOTHING: VEDL
1
0
strategy 1, expansion breakout = NOTHING: ULTRACEMCO
st

strategy 1, expansion breakout = NOTHING: IOC
strategy 2, OTT pullback = NOTHING: IOC
strategy 3, expansion pivots = NOTHING: IOC
strategy 4, 180 deg = NOTHING: IOC
strategy 5, gulligan island = NOTHING: IOC
strategy 6, Boomer = NOTHING: IOC
strategy 7, Slingshot = NOTHING: IOC
strategy 8, Whoops = NOTHING: IOC
strategy 9, Lizards = NOTHING: IOC
0
0
strategy 1, expansion breakout = NOTHING: ITC
strategy 3, expansion pivots = NOTHING: ITC
strategy 4, 180 deg = NOTHING: ITC
strategy 5, gulligan island = NOTHING: ITC
strategy 7, Slingshot = NOTHING: ITC
strategy 8, Whoops = NOTHING: ITC
strategy 9, Lizards = NOTHING: ITC
0
0
strategy 1, expansion breakout = NOTHING: ICICIBANK
strategy 2, OTT pullback = NOTHING: ICICIBANK
strategy 3, expansion pivots = NOTHING: ICICIBANK
strategy 4, 180 deg = NOTHING: ICICIBANK
strategy 5, gulligan island = NOTHING: ICICIBANK
strategy 6, Boomer = NOTHING: ICICIBANK
strategy 7, Slingshot = NOTHING: ICICIBANK
strategy 8, Whoops = NOTHING: ICICIBANK
0
0
strat

strategy 1, expansion breakout = NOTHING: IDEA
strategy 2, OTT pullback = NOTHING: IDEA
strategy 3, expansion pivots = NOTHING: IDEA
strategy 5, gulligan island = NOTHING: IDEA
strategy 6, Boomer = NOTHING: IDEA
strategy 7, Slingshot = NOTHING: IDEA
strategy 8, Whoops = NOTHING: IDEA
strategy 9, Lizards = NOTHING: IDEA
0
0
strategy 1, expansion breakout = NOTHING: MCDOWELL-N
strategy 2, OTT pullback = NOTHING: MCDOWELL-N
strategy 3, expansion pivots = NOTHING: MCDOWELL-N
strategy 5, gulligan island = NOTHING: MCDOWELL-N
strategy 6, Boomer = NOTHING: MCDOWELL-N
strategy 7, Slingshot = NOTHING: MCDOWELL-N
strategy 8, Whoops = NOTHING: MCDOWELL-N
strategy 9, Lizards = NOTHING: MCDOWELL-N
0
0
strategy 1, expansion breakout = NOTHING: UBL
strategy 2, OTT pullback = NOTHING: UBL
strategy 3, expansion pivots = NOTHING: UBL
strategy 4, 180 deg = NOTHING: UBL
strategy 5, gulligan island = NOTHING: UBL
strategy 6, Boomer = NOTHING: UBL
strategy 7, Slingshot = NOTHING: UBL
strategy 8, Whoops = NO

strategy 1, expansion breakout = NOTHING: HINDZINC
strategy 3, expansion pivots = NOTHING: HINDZINC
strategy 4, 180 deg = NOTHING: HINDZINC
strategy 5, gulligan island = NOTHING: HINDZINC
strategy 7, Slingshot = NOTHING: HINDZINC
strategy 8, Whoops = NOTHING: HINDZINC
0
0
strategy 1, expansion breakout = NOTHING: HINDPETRO
strategy 2, OTT pullback = NOTHING: HINDPETRO
strategy 3, expansion pivots = NOTHING: HINDPETRO
strategy 5, gulligan island = NOTHING: HINDPETRO
strategy 6, Boomer = NOTHING: HINDPETRO
strategy 7, Slingshot = NOTHING: HINDPETRO
strategy 8, Whoops = NOTHING: HINDPETRO
strategy 9, Lizards = NOTHING: HINDPETRO
0
0
strategy 1, expansion breakout = NOTHING: HAVELLS
RARE strategy 2, 1-2-3 pullback = buy: HAVELLS
strategy 3, expansion pivots = NOTHING: HAVELLS
strategy 5, gulligan island = NOTHING: HAVELLS
strategy 7, Slingshot = NOTHING: HAVELLS
strategy 8, Whoops = NOTHING: HAVELLS
1
0
strategy 1, expansion breakout = NOTHING: HDFCLIFE
RARE strategy 2, 1-2-3 pullback = bu

strategy 1, expansion breakout = NOTHING: WABCOINDIA
strategy 3, expansion pivots = NOTHING: WABCOINDIA
strategy 4, 180 deg = NOTHING: WABCOINDIA
strategy 5, gulligan island = NOTHING: WABCOINDIA
strategy 6, Boomer = buy: WABCOINDIA
strategy 7, Slingshot = NOTHING: WABCOINDIA
strategy 8, Whoops = NOTHING: WABCOINDIA
strategy 9, Lizards = NOTHING: WABCOINDIA
1
0
strategy 1, expansion breakout = NOTHING: VOLTAS
strategy 2, OTT pullback = NOTHING: VOLTAS
strategy 3, expansion pivots = NOTHING: VOLTAS
strategy 4, 180 deg = NOTHING: VOLTAS
strategy 5, gulligan island = NOTHING: VOLTAS
strategy 6, Boomer = NOTHING: VOLTAS
strategy 7, Slingshot = NOTHING: VOLTAS
strategy 8, Whoops = NOTHING: VOLTAS
0
0
strategy 1, expansion breakout = NOTHING: VINATIORGA
strategy 2, OTT pullback = NOTHING: VINATIORGA
strategy 3, expansion pivots = NOTHING: VINATIORGA
strategy 5, gulligan island = NOTHING: VINATIORGA
strategy 6, Boomer = NOTHING: VINATIORGA
strategy 7, Slingshot = NOTHING: VINATIORGA
strategy 

...

strategy 1, expansion breakout = NOTHING: INDIANB
RARE strategy 2, 1-2-3 pullback = buy: INDIANB
strategy 3, expansion pivots = NOTHING: INDIANB
strategy 4, 180 deg = NOTHING: INDIANB
strategy 5, gulligan island = NOTHING: INDIANB
strategy 7, Slingshot = NOTHING: INDIANB
strategy 8, Whoops = NOTHING: INDIANB
strategy 9, Lizards = NOTHING: INDIANB
1
0
strategy 1, expansion breakout = NOTHING: IBVENTURES
RARE strategy 2, 1-2-3 pullback = buy: IBVENTURES
strategy 3, expansion pivots = NOTHING: IBVENTURES
strategy 4, 180 deg = NOTHING: IBVENTURES
strategy 5, gulligan island = NOTHING: IBVENTURES
strategy 7, Slingshot = NOTHING: IBVENTURES
strategy 8, Whoops = NOTHING: IBVENTURES
strategy 9, Lizards = NOTHING: IBVENTURES
1
0
strategy 1, expansion breakout = NOTHING: IDFCFIRSTB
strategy 3, expansion pivots = NOTHING: IDFCFIRSTB
strategy 4, 180 deg = NOTHING: IDFCFIRSTB
strategy 5, gulligan island = NOTHING: IDFCFIRSTB
strategy 7, Slingshot = NOTHING: IDFCFIRSTB
strategy 8, Whoops = NOTHING: 

strategy 1, expansion breakout = NOTHING: ESCORTS
RARE strategy 2, 1-2-3 pullback = buy: ESCORTS
strategy 3, expansion pivots = NOTHING: ESCORTS
strategy 4, 180 deg = buy: ESCORTS
strategy 5, gulligan island = NOTHING: ESCORTS
strategy 7, Slingshot = NOTHING: ESCORTS
strategy 8, Whoops = NOTHING: ESCORTS
strategy 9, Lizards = NOTHING: ESCORTS
2
0
strategy 1, expansion breakout = NOTHING: ERIS
strategy 2, OTT pullback = NOTHING: ERIS
strategy 4, 180 deg = NOTHING: ERIS
strategy 5, gulligan island = NOTHING: ERIS
strategy 6, Boomer = NOTHING: ERIS
strategy 7, Slingshot = NOTHING: ERIS
strategy 8, Whoops = NOTHING: ERIS
strategy 9, Lizards = NOTHING: ERIS
0
0
strategy 1, expansion breakout = NOTHING: ENGINERSIN
strategy 3, expansion pivots = NOTHING: ENGINERSIN
strategy 5, gulligan island = NOTHING: ENGINERSIN
strategy 7, Slingshot = buy: ENGINERSIN
strategy 8, Whoops = NOTHING: ENGINERSIN
strategy 9, Lizards = NOTHING: ENGINERSIN
1
0
strategy 1, expansion breakout = NOTHING: ENDURANCE
RA

strategy 1, expansion breakout = NOTHING: BATAINDIA
strategy 2, OTT pullback = NOTHING: BATAINDIA
strategy 3, expansion pivots = NOTHING: BATAINDIA
strategy 4, 180 deg = NOTHING: BATAINDIA
strategy 5, gulligan island = NOTHING: BATAINDIA
strategy 6, Boomer = NOTHING: BATAINDIA
strategy 7, Slingshot = NOTHING: BATAINDIA
strategy 8, Whoops = NOTHING: BATAINDIA
strategy 9, Lizards = NOTHING: BATAINDIA
0
0
strategy 1, expansion breakout = NOTHING: BANKINDIA
RARE strategy 2, 1-2-3 pullback = buy: BANKINDIA
strategy 3, expansion pivots = NOTHING: BANKINDIA
strategy 4, 180 deg = NOTHING: BANKINDIA
strategy 5, gulligan island = NOTHING: BANKINDIA
strategy 7, Slingshot = NOTHING: BANKINDIA
strategy 8, Whoops = NOTHING: BANKINDIA
strategy 9, Lizards = NOTHING: BANKINDIA
1
0
strategy 1, expansion breakout = NOTHING: BALKRISIND
RARE strategy 2, 1-2-3 pullback = buy: BALKRISIND
strategy 3, expansion pivots = NOTHING: BALKRISIND
strategy 5, gulligan island = NOTHING: BALKRISIND
strategy 7, Slingshot

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:124: RuntimeWarning: invalid value encountered in double_scalars
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:130: RuntimeWarning: invalid value encountered in double_scalars
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:137: RuntimeWarning: invalid value encountered in double_scalars
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:144: RuntimeWarning: invalid value encountered in double_scalars
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:212: RuntimeWarning: invalid value encountered in double_scalars
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:217: RuntimeWarning: invalid value encountered in double_scalars


strategy 1, expansion breakout = NOTHING: ADANIGAS
RARE strategy 2, 1-2-3 pullback = buy: ADANIGAS
strategy 3, expansion pivots = NOTHING: ADANIGAS
strategy 4, 180 deg = NOTHING: ADANIGAS
strategy 5, gulligan island = NOTHING: ADANIGAS
strategy 7, Slingshot = NOTHING: ADANIGAS
strategy 8, Whoops = NOTHING: ADANIGAS
strategy 9, Lizards = NOTHING: ADANIGAS
1
0
strategy 1, expansion breakout = NOTHING: ABBOTINDIA
strategy 2, OTT pullback = NOTHING: ABBOTINDIA
strategy 3, expansion pivots = NOTHING: ABBOTINDIA
strategy 5, gulligan island = NOTHING: ABBOTINDIA
strategy 6, Boomer = NOTHING: ABBOTINDIA
strategy 7, Slingshot = NOTHING: ABBOTINDIA
strategy 8, Whoops = NOTHING: ABBOTINDIA
strategy 9, Lizards = NOTHING: ABBOTINDIA
0
0
strategy 1, expansion breakout = NOTHING: AAVAS
strategy 2, OTT pullback = NOTHING: AAVAS
strategy 3, expansion pivots = NOTHING: AAVAS
strategy 5, gulligan island = NOTHING: AAVAS
strategy 6, Boomer = NOTHING: AAVAS
strategy 7, Slingshot = NOTHING: AAVAS
strategy 8